In [ ]:
#works with cv virtual environment

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

import sys
import glob
import cv2

import pickle

In [2]:
# import sys
# import glob
# import os
# import cv2
# import numpy as np

# from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D
# from keras.models import Model, load_model
# from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# from keras import backend as K
# import keras

In [3]:
folder_path = '/home/niaki/Code/hpatches/hpatches-benchmark/data/hpatches-release'
patch_size = 65

In [30]:
# all types of patches 
tps = ['ref','e1','e2','e3','e4','e5','h1','h2','h3','h4','h5',\
       't1','t2','t3','t4','t5']

class hpatches_sequence:
    """Class for loading an HPatches sequence from a sequence folder"""
    itr = tps
    def __init__(self,base):
        name = base.split('/')
        self.name = name[-1]
        self.base = base
        for t in self.itr:
            im_path = os.path.join(base, t+'.png')
            im = cv2.imread(im_path,0)
#             print(im.shape)
#             print(im[0,0])
#             print(type(im[0,0]))
            im_imageio = imageio.imread(im_path)
#             print(im_imageio.shape)
#             print(im_imageio[0,0])
#             print(type(im_imageio[0,0]))
            print(np.array_equal(im, im_imageio))
#             print()
            
            self.N = im.shape[0]/patch_size
            setattr(self, t, np.split(im, self.N))

In [5]:
seqs = glob.glob(folder_path+'/*')
seqs = [os.path.abspath(p) for p in seqs]

In [6]:
len(seqs)

116

In [7]:
def unpickle_vars(pickle_file_path):
    try:
        variables = pickle.load(open(pickle_file_path, "rb"))
        return variables
    except Exception as e:
        print("Problem while trying to unpickle: ", str(e))
        return None

In [8]:
descr_name = 'vae_0.0.0.4'
base_dir = '/home/niaki/Code/sundry_segments/weights'
model_version = '0.0.0.4_dataaug_300moreepoch'
vae_patch_size = 56

In [9]:
latent_dim = 128

In [10]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [11]:
encoder_weights = unpickle_vars('zimnica/pickled_vae_encoder_0.0.0.4_20200713_114306.pickle')

In [12]:
decoder_weghts = unpickle_vars('zimnica/pickled_vae_decoder_0.0.0.4_20200713_114306.pickle')

In [13]:
init_kernel = encoder_weights[0]
init_bias = encoder_weights[1]
kernel_initializer = tf.keras.initializers.constant(init_kernel)
bias_initializer = tf.keras.initializers.constant(init_bias)

In [14]:
class Encoder(layers.Layer):
    """Maps input patches to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.conv1 = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")
        self.conv2 = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")
        self.conv3 = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(256, activation="relu")
        self.dense_mean = layers.Dense(latent_dim, name="z_mean")
        self.dense_log_var = layers.Dense(latent_dim, name="z_log_var")
        self.sampling = Sampling()
        

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense(x)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))        
        return z_mean, z_log_var, z

In [15]:
class Decoder(layers.Layer):
    """Converts z, the encoded patch vector, back into the patch."""

    def __init__(self, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.dense = layers.Dense(7 * 7 * 64, activation="relu")
        self.reshape = layers.Reshape((7, 7, 64))
        self.conv1 = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")
        self.conv2 = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")
        self.conv3 = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")
        self.conv_output = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")
        
    def call(self, inputs):
        x = self.dense(inputs)
        x = self.reshape(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv_output(x)
        return x

In [16]:
class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        name="vae",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.encoder = Encoder()
        self.decoder = Decoder()

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [17]:
vae = VariationalAutoEncoder()

In [2]:
tf.__version__

'1.4.0'

In [22]:
fake_batch = np.ones((5, 56, 56, 1), dtype=np.float32)
# fake_batch = tf.data.Dataset.from_tensor_slices(fake_batch)

In [27]:
len(zip(fake_batch))

5

In [28]:
fake_batch_reconstructed = vae(zip(fake_batch))

ValueError: Layer conv2d expects 1 inputs, but it received 5 input tensors. Inputs received: [array([[[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       ...,

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]]], dtype=float32), array([[[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
  

In [18]:
vae.load_weights(base_dir + '/vae_' + model_version + '.h5')

ValueError: You are trying to load a weight file containing 2 layers into a model with 0 layers.

In [48]:
vae.encoder.conv1.get_weights()

[]

In [29]:
print(encoder.output_shape)

[(None, 128), (None, 128), (None, 128)]


In [ ]:
# for seq_path in seqs:
#     seq = hpatches_sequence(seq_path)
#     path = os.path.join(descr_name,seq.name)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     descr = np.zeros((seq.N,2)) # trivial (mi,sigma) descriptor
#     for tp in tps:
#         print(seq.name+'/'+tp)
#         for i,patch in enumerate(getattr(seq, tp)):
#             mi = np.mean(patch) # trivial (mi,sigma) descriptor
#             sigma = np.std(patch) # trivial (mi,sigma) descriptor
#             descr[i] = np.array([mi,sigma])
#         np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

In [34]:
path

'vae_0.0.0.2/i_kurhaus'

In [37]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join("hpatches_extracted_descrs", descr_name, seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = 128
    descr = np.zeros((seq.N,descr_size_per_patch)) # trivial (mi,sigma) descriptor   
    print(seq.name)
    for tp in tps:
#         print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
            patch = patch / 255.0
#             print(patch)
            patch_crop = patch[4: 60, 4: 60]
            patch_crop_encoded = encoder.predict(np.expand_dims(np.expand_dims(patch_crop, axis=-1), axis=0))
            patch_crop_encoded_flat = patch_crop_encoded[2].flatten()
#             print(patch_crop.shape)
#             print(patch_crop_encoded[2].shape)
#             print(patch_crop_encoded_flat.shape)
#             print()
            descr[i] = patch_crop_encoded_flat
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

i_kurhaus
i_veggies
i_fruits
v_grace
v_bark
i_boutique
v_tempera
i_dome
v_busstop
v_vitro
v_bip
v_strand
i_salon
i_brooklyn
i_steps
v_samples
v_dirtywall
i_bologna
i_fenis
v_wormhole
v_posters
i_londonbridge
i_indiana
i_objects
i_nijmegen
v_cartooncity
i_fog
v_yard
i_tools
i_nescafe
i_gonnenberg
v_gardens
i_lionnight
v_home
i_autannes
i_whitebuilding
v_wall
i_smurf
v_yuri
i_kions
i_greentea
i_pinard
v_bricks
v_charing
v_artisans
v_there
v_wounded
v_soldiers
v_bees
i_zion
i_porta
v_fest
v_calder
v_machines
i_yellowtent
v_sunseason
i_resort
i_leuven
i_crownday
i_village
v_dogman
i_ktirio
v_blueprint
i_ajuntament
i_bridger
i_greenhouse
v_london
i_books
i_chestnuts
v_courses
v_boat
v_coffeehouse
i_crownnight
i_toy
v_astronautis
i_dc
i_nuts
v_abstract
i_ski
i_santuario
v_beyus
v_tabletop
v_underground
v_feast
i_lionday
v_weapons
v_bird
v_maskedman
i_table
v_adam
i_parking
v_laptop
v_churchill
i_pool
i_castle
i_melon
i_contruction
v_wapping
v_talent
i_miniature
v_man
v_apprentices
v_eastsout

In [31]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 